In [10]:
import re

In [11]:
OTHER = ' '
DIACRITICS = [OTHER, "َ", "ً", "ُ", "ٌ", "ِ", "ٍ", "ْ", "ّ", "َّ", "ًّ", "ُّ", "ٌّ", "ِّ", "ٍّ"]
VOWEL_REGEX = re.compile('|'.join(DIACRITICS))
SENTENCE_WINDOW = 800
WINDOW_SIZE_BEFORE = 2
WINDOW_SIZE_AFTER = 3

MAIN_DIACRITICS = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
PUNCTUATIONS = [".", "،", ":", "؛", "؟"]
SPLITTING_PATTERN = re.compile(r"[\.،:؛!؟]")

In [12]:
def readFile(path):
	sentences = []
	with open(path, 'r', encoding='utf-8') as file:
		for line in file:
			sentences.append(line.strip())

	return sentences

In [13]:
'''
Getting Basic Arabic Letters based on their unicodes
'''
basic_arabic_start1 = 0x0621
basic_arabic_end1 = 0x063A
basic_arabic_start2 = 0x0641
basic_arabic_end2 = 0x064A

basic_arabic_letters = [chr(code) for code in range(basic_arabic_start1, basic_arabic_end1 + 1)] + \
                       [chr(code) for code in range(basic_arabic_start2, basic_arabic_end2 + 1)]

VALID_ARABIC_CHARS = basic_arabic_letters + MAIN_DIACRITICS + [' '] + PUNCTUATIONS
VALID_ARABIC_CHARS_WITHOUT_PUNCTUATION = basic_arabic_letters + MAIN_DIACRITICS + [' ']

In [14]:
char_to_index = {char: i + 1 for i, char in enumerate(basic_arabic_letters)}
diacritic_to_index = {diacritic: i for i, diacritic in enumerate(DIACRITICS)}
index_to_diacritic = {index: diacritic for diacritic, index in diacritic_to_index.items()}

In [15]:
WHITESPACES_PATTERN = re.compile("\s+")
def combine_whitespaces(text):
    return re.sub(WHITESPACES_PATTERN, " ",text).strip()

In [16]:
def get_valid_arabic_text(text):
    text = list(filter(lambda char: char in VALID_ARABIC_CHARS,text))
    return combine_whitespaces(''.join(text))

In [17]:
def get_valid_arabic_text_without_punctuation(text):
    text = list(filter(lambda char: char in VALID_ARABIC_CHARS_WITHOUT_PUNCTUATION,text))
    return combine_whitespaces(''.join(text))

In [18]:
def get_sentences_window(sentence):
    #Fel bdaya ehna 3ayzeen n2smha 600 char 600 char
    startIdx = 0
    sentences = []
    sentence = get_valid_arabic_text_without_punctuation(sentence)
    while(startIdx < len(sentence)):
        finalIdx = startIdx + SENTENCE_WINDOW if startIdx + SENTENCE_WINDOW < len(sentence) else -1
        pre_modified_sentence = sentence[startIdx:finalIdx]
        startIdx+=SENTENCE_WINDOW
        while(len(pre_modified_sentence) > 0 and pre_modified_sentence[-1]!=' '):
            pre_modified_sentence = pre_modified_sentence[0:-2]
            startIdx-=1
        sentences.append(pre_modified_sentence)
    return sentences

In [19]:
def get_splitted_sentences(sentence):
    sentence = get_valid_arabic_text(sentence)
    #return [item.strip() for item in re.split(SPLITTING_PATTERN, sentence)]
    return get_sentences_window(sentence)

In [20]:
def get_all_windows(sentence):
    windows =[]
    sentence = get_valid_arabic_text_without_punctuation(sentence)
    sentence = sentence.split()
    for i in range(len(sentence)):
        start_idx = max(0,i-WINDOW_SIZE_BEFORE)
        last_index = min(len(sentence),i+WINDOW_SIZE_AFTER+1)
        windows.append(sentence[start_idx:last_index])
    return windows

In [21]:
def separate_words_and_diacritics(sentence):
    sentences = get_splitted_sentences(sentence)
    
    final_chars = []
    final_diacritics = []

    for sentence in sentences:
        output_chars = []
        output_diacritics = []
        for word in sentence.split():
            letters = []
            diacritics = []
            prev_char = word[0]
            for idx, char in enumerate(word[1:]):
                try:
                    next_char = word[idx + 1 + 1]
                except IndexError:
                    next_char = ''
                if char in DIACRITICS:
                    if prev_char not in DIACRITICS:
                        letters.append(prev_char)
                        if next_char == '' or next_char not in DIACRITICS:
                            diacritics.append(char)
                        elif next_char in DIACRITICS:
                            diacritics.append(char + next_char)
                else:
                    if prev_char not in DIACRITICS:
                        letters.append(prev_char)
                        diacritics.append(OTHER)
                    if next_char == '':
                        letters.append(char)
                        diacritics.append(OTHER)
                prev_char = char

            if len(letters):
                output_chars.append(letters)
                output_diacritics.append(diacritics)

        final_chars.append([char for word in output_chars for char in word])
        final_diacritics.append([diacritic for word in output_diacritics for diacritic in word])

    final_chars = [item for item in final_chars if len(item)]
    final_diacritics = [item for item in final_diacritics if len(item)]
    return final_chars, final_diacritics

In [25]:
def separate_words_to_char(sentence):
    sentence = get_valid_arabic_text_without_punctuation(sentence)

    letters = []
    for word in sentence.split():
        for char in word:
            letters.append(char)

    return letters

In [23]:
def preprocess():
    sentence = ("( قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ ) أَيْ الْوَصِيَّةُ ( قَوْلُهُ مَا مَرَّ ) أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَبْطُلُ ، وَعِبَارَةُ الْكَنْزِ وَلَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي لَمْ يَتَعَيَّنْ غَنَمُهُ إنْ كَانَتْ انْتَهَتْ سم ( قَوْلُهُ فَيُعْطَى وَاحِدَةً مِنْهَا إلَخْ ) كَمَا لَوْ كَانَتْ مَوْجُودَةً عِنْدَ الْوَصِيَّةِ وَالْمَوْتِ ، وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ فِي الصُّورَتَيْنِ وَإِنْ تَرَاضَيَا ؛ لِأَنَّهُ صُلْحٌ عَلَى مَجْهُولٍ مُغْنِي وَنِهَايَةٌ قَالَ ع ش قَوْلُهُ وَاحِدَةً مِنْهَا أَيْ كَامِلَةً ، وَلَا يَجُوزُ أَنْ يُعْطَى نِصْفَيْنِ مِنْ شَاتَيْنِ ؛ لِأَنَّهُ لَا يُسَمَّى شَاةً وَقَوْلُهُ وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ.")
    
    char, dia = separate_words_and_diacritics(sentence)
    sentences = get_splitted_sentences(sentence)
    windows = get_all_windows(sentence)
    
    for window in windows:
        print(window)
        print("_________________")
        
    for splitted_sentence in sentences:
        print(splitted_sentence)
        print(len(splitted_sentence))
        print("*****************")
        
    for i in range(len(char)):
        print(len(char[i]))
        print(char[i])
        print("-----------------")

474
['ق', 'و', 'ل', 'ه', 'ل', 'ع', 'د', 'م', 'م', 'ا', 'ت', 'ت', 'ع', 'ل', 'ق', 'إ', 'ل', 'خ', 'أ', 'ي', 'ا', 'ل', 'و', 'ص', 'ي', 'ة', 'ق', 'و', 'ل', 'ه', 'م', 'ا', 'م', 'ر', 'أ', 'ي', 'ق', 'ب', 'ي', 'ل', 'ق', 'و', 'ل', 'ا', 'ل', 'م', 'ت', 'ن', 'ل', 'غ', 'ت', 'و', 'ل', 'و', 'ا', 'ق', 'ت', 'ص', 'ر', 'ع', 'ل', 'ى', 'أ', 'و', 'ص', 'ي', 'ت', 'ل', 'ه', 'ب', 'ش', 'ا', 'ة', 'أ', 'و', 'أ', 'ع', 'ط', 'و', 'ه', 'ش', 'ا', 'ة', 'و', 'ل', 'ا', 'غ', 'ن', 'م', 'ل', 'ه', 'ع', 'ن', 'د', 'ا', 'ل', 'م', 'و', 'ت', 'ه', 'ل', 'ت', 'ب', 'ط', 'ل', 'ا', 'ل', 'و', 'ص', 'ي', 'ة', 'أ', 'و', 'ي', 'ش', 'ت', 'ر', 'ى', 'ل', 'ه', 'ش', 'ا', 'ة', 'و', 'ي', 'ؤ', 'خ', 'ذ', 'م', 'ن', 'ق', 'و', 'ل', 'ه', 'ا', 'ل', 'آ', 'ت', 'ي', 'ك', 'م', 'ا', 'ل', 'و', 'ل', 'م', 'ي', 'ق', 'ل', 'م', 'ن', 'م', 'ا', 'ل', 'ي', 'و', 'ل', 'ا', 'م', 'ن', 'غ', 'ن', 'م', 'ي', 'أ', 'ن', 'ه', 'ا', 'ل', 'ا', 'ت', 'ب', 'ط', 'ل', 'و', 'ع', 'ب', 'ا', 'ر', 'ة', 'ا', 'ل', 'ك', 'ن', 'ز', 'و', 'ل', 'و', 'ل', 'م', 'ي', 'ق', 'ل', 'م', 'ن', 'م', 'ا', 'ل', 'ي', 